In [1]:
import collections
from csv import DictReader
from collections import defaultdict
import numpy as np
import matplotlib.pyplot as plt

In [2]:
season16_17 = DictReader(open('podatki/season-1617_csv.csv', 'rt', encoding='utf-8'))
season17_18 = DictReader(open('podatki/season-1718_csv.csv', 'rt', encoding='utf-8'))
season18_19 = DictReader(open('podatki/season-1819_csv.csv', 'rt', encoding='utf-8'))

In [3]:
class Referee:
    '''
    Podatki na voljo v zvezi s sodnikom:
        Referee = Match Referee
        HF = Home Team Fouls Committed
        AF = Away Team Fouls Committed
        HO = Home Team Offsides
        AO = Away Team Offsides
        HY = Home Team Yellow Cards
        AY = Away Team Yellow Cards
        HR = Home Team Red Cards
        AR = Away Team Red Cards
        HBP = Home Team Bookings Points (10 = yellow, 25 = red)
        ABP = Away Team Bookings Points (10 = yellow, 25 = red)
    '''

    # dict  {ime priimek: objekt}
    # dict[imepriimek].refereed_games

    def __init__(self, name):
        # Ime sodnika v formatu "'Prva črka imena' Priimek"

        self.name = name

        # Sojene tekme: tekme v katerih je bil glaven sodnik
        # Dodajajo se objekti "Match"
        self.refereed_games = []

        # Rumeni kartoni (razdeljeni na domače/goste):
        self.yellow_cards_home = 0
        self.yellow_cards_away = 0

        # Rdeči kartoni:
        self.red_cards_home = 0
        self.red_cards_away = 0

        # Skupno število točk od kartonov (10 = rumen, 25 = rdeč)
        self.booking_points = 0

        #Sojeni prekrški
        self.home_team_fouls = 0
        self.away_team_fouls = 0

    def add_all(self, cards, match):
        self.add_cards(cards)
        self.add_match(match)
        
    def add_cards(self, cards):
        # cards seznam kartonov v formatu [hy, ay, hr, ar]
        hy = int(cards[0])
        ay = int(cards[1])
        hr = int(cards[2])
        ar = int(cards[3])
        self.yellow_cards_home += hy
        self.yellow_cards_away += ay
        self.red_cards_home += hr
        self.red_cards_away += ar

        self.booking_points += (hy + ay) * 10 + (hr + ar) * 25

    def add_match(self, match):
        self.refereed_games.append(match)

    def to_string(self):
        return(self.name)
    def get_booking_points(self):
        return self.booking_points
    
    
    
class Match:
    '''
     Podatki na voljo:
        Div = League Division
        Date = Match Date (dd/mm/yy)
        HomeTeam = Home Team
        AwayTeam = Away Team
        FTHG = Full Time Home Team Goals
        FTAG = Full Time Away Team Goals
        FTR = Full Time Result (H=Home Win, D=Draw, A=Away Win)
        HTHG = Half Time Home Team Goals
        HTAG = Half Time Away Team Goals
        HTR = Half Time Result (H=Home Win, D=Draw, A=Away Win)

        Match Statistics (where available)
        Attendance = Crowd Attendance
        Referee = Match Referee
        HTS = Home Team Shots
        ATS = Away Team Shots
        HST = Home Team Shots on Target
        AST = Away Team Shots on Target
        HHW = Home Team Hit Woodwork
        AHW = Away Team Hit Woodwork
        HC = Home Team Corners
        AC = Away Team Corners
        HF = Home Team Fouls Committed
        AF = Away Team Fouls Committed
        HO = Home Team Offsides
        AO = Away Team Offsides
        HY = Home Team Yellow Cards
        AY = Away Team Yellow Cards
        HR = Home Team Red Cards
        AR = Away Team Red Cards
    '''

    def __init__(self, date, home_team, away_team, fthg, ftag, ftr, hthg, 
                 htr, hts, ats, hst, ast, hc, ac, hf, af, hy, ay, hr, ar):
        self.date = date
        self.home_team = home_team
        self.away_team = away_team
        self.fthg = fthg
        self.ftag = ftag
        self.ftr = ftr
        self.hthg = hthg
        self.htr = htr
        self.hts = hts
        self.ats = ats
        self.hst = hst
        self.ast = ast
        self.hc = hc
        self.ac = ac
        self.hf = hf
        self.af = af
        self.hy = hy
        self.ay = ay
        self.hr = hr
        self.ar = ar


    def to_string(self):
        out = "{} {}{} - {}{}".format(self.date, self.home_team, self.fthg, self.ftag, self.away_team)
        return out


class Team:
    '''
        Div = League Division
        Date = Match Date (dd/mm/yy)
        FTHG = Full Time Home Team Goals
        FTR = Full Time Result (H=Home Win, D=Draw, A=Away Win)
        HTHG = Half Time Home Team Goals
        HTR = Half Time Result (H=Home Win, D=Draw, A=Away Win)

        Match Statistics (where available)
        Attendance = Crowd Attendance
        Referee = Match Referee
        HS = Home Team Shots
        HST = Home Team Shots on Target
        HHW = Home Team Hit Woodwork
        HC = Home Team Corners
        HF = Home Team Fouls Committed
        HO = Home Team Offsides
        HY = Home Team Yellow Cards
        HR = Home Team Red Cards
        HBP = Home Team Bookings Points (10 = yellow, 25 = red)


    '''


    def __init__(self, name, division, ):
        self.name = name
        self.division = division
        self.goals_scored = 0
        self.goals_conceded = 0
        self.yellow_cards = 0
        self.red_cards = 0
        self.points = 0
        
    def add_all(self, gs, gc, yc, rc):
        self.goals_scored += int(gs)
        self.goals_conceded += int(gc)
        self.yellow_cards += int(yc)
        self.red_cards += int(rc)
        
        if gs > gc:
            self.points += 3
        if gs == gc:
            self.points += 1        
        
    def to_string(self):
        return "{} {}".format(self.name, self.division)

    


In [4]:
sezone_referees = []
sezone_teams = []
#referee_dict = {}
# {"Ime ekipe": objekt}
# teams_dict = defaultdict(Team)
#teams_dict = {}

def read_data(data):
    referee_dict = {}
    teams_dict = {}
    for row in data:
        div = row["Div"]
        date = row["Date"]
        home_team = row["HomeTeam"]
        away_team = row["AwayTeam"]
        #home_hit_woodwork = row["HHW"]
        #away_hit_woodwork = row["AHW"]
        full_time_home_goals = row["FTHG"]
        full_time_away_goals = row["FTAG"]
        full_time_result = row["FTR"]
        half_time_home_goals = row["HTHG"]
        htag = row["HTAG"]
        half_time_result = row["HTR"]
        referee = row["Referee"]
        hts = row["HS"]
        ats = row["AS"]
        hst = row["HST"]
        ast = row["AST"]
        home_fouls = row["HF"]
        af = row["AF"]
        #home_team_offsides = row["HO"]
        #away_team_offsides = row["AO"]
        hc = row["HC"]
        ac = row["AC"]
        home_yellow = row["HY"]
        away_yellow = row["AY"]
        home_red = row["HR"]
        away_red = row["AR"]

        # print(referee)
        # print(curr_referee.to_string())
        curr_referee = ""
        curr_match = ""

        try:
            curr_referee = referee_dict[referee]
        except KeyError:
            curr_referee = Referee(referee)
            referee_dict[referee] = curr_referee

        curr_match = Match(date, home_team, away_team, full_time_home_goals, full_time_away_goals, full_time_result,
                           half_time_home_goals, half_time_result, hts, ats,
                           hst, ast, hc, ac, home_fouls, af, home_yellow, away_yellow, home_red, away_red)

        referee_dict[referee].add_all([home_yellow, away_yellow, home_red, away_red], curr_match)
        curr_home_team = curr_away_team = ""
        
        try:
            curr_home_team = teams_dict[home_team]
            curr_away_team = teams_dict[away_team]
        except KeyError:
            curr_home_team = Team(home_team, div)
            curr_away_team = Team(away_team, div)

            teams_dict[home_team] = curr_home_team
            teams_dict[away_team] = curr_away_team

        curr_home_team.add_all(full_time_home_goals, full_time_away_goals,
                               home_yellow, home_red)
        curr_away_team.add_all(full_time_away_goals, full_time_home_goals,
                               away_yellow, away_red)
        
    return(referee_dict, teams_dict)

    
referees, teams = read_data(season16_17)
sezone_referees.append(referees)
sezone_teams.append(teams)

referees, teams = read_data(season17_18)
sezone_referees.append(referees)
sezone_teams.append(teams)

referees, teams = read_data(season18_19)
sezone_referees.append(referees)
sezone_teams.append(teams)
# print(referee_dict.items())

[{'Burnley': <__main__.Team object at 0x0336DF70>, 'Swansea': <__main__.Team object at 0x0336DF88>, 'Crystal Palace': <__main__.Team object at 0x03363028>, 'West Brom': <__main__.Team object at 0x03363040>, 'Everton': <__main__.Team object at 0x03363088>, 'Tottenham': <__main__.Team object at 0x033630A0>, 'Hull': <__main__.Team object at 0x033630E8>, 'Leicester': <__main__.Team object at 0x03363100>, 'Man City': <__main__.Team object at 0x03363148>, 'Sunderland': <__main__.Team object at 0x03363160>, 'Middlesbrough': <__main__.Team object at 0x033631A8>, 'Stoke': <__main__.Team object at 0x033631C0>, 'Southampton': <__main__.Team object at 0x03363208>, 'Watford': <__main__.Team object at 0x03363220>, 'Arsenal': <__main__.Team object at 0x03363268>, 'Liverpool': <__main__.Team object at 0x03363280>, 'Bournemouth': <__main__.Team object at 0x033632C8>, 'Man United': <__main__.Team object at 0x033632E0>, 'Chelsea': <__main__.Team object at 0x03363328>, 'West Ham': <__main__.Team object at

In [5]:
import matplotlib.pyplot as plt
